<a href="https://colab.research.google.com/github/krvicky/open_spiel/blob/main/CFR_and_MCCFR_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
!pip install open-spiel  # This might take some time

In [ ]:
# Import necessary libraries
from open_spiel.python.algorithms import cfr
from open_spiel.python.algorithms import exploitability
from open_spiel.python.algorithms import external_sampling_mccfr as external_mccfr
from open_spiel.python.algorithms import outcome_sampling_mccfr as outcome_mccfr
import pyspiel
import pandas as pd
from datetime import datetime

In [ ]:
#Create empty output dataframe
data = {
    'Experiment number': [],
    'Algorithm': [],
    'Iteration number': [],
    'Time stamp': [],
    'Exploitability': []
}

output_df = pd.DataFrame(data)

In [ ]:
#FUNCTION TO UPDATE THE OUTPUT DF
def output_df_update(experiment_number, algorithm, iteration_number, exploitability, output_df = output_df):
  # Create list of data to be added to the DataFrame
  data = {
      'Experiment number': experiment_number,
      'Algorithm': algorithm,
      'Iteration number': iteration_number,
      'Time stamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
      'Exploitability': exploitability
  }

  # Append data to the dataframe
  output_df = pd.concat([output_df, pd.DataFrame(data, index=[0])], ignore_index=True)

  return output_df

In [ ]:
#write the function to run the experiment
def cfr_comparison(game = "kuhn_poker", iterations = 10000, output_df = output_df):

  #FIND EXPERIMENT NUMBER
  if not output_df.empty:
    max_experiment_number = output_df['Experiment number'].max()
  else:
    max_experiment_number = 0
  experiment_number = max_experiment_number + 1

  print("Starting Vanilla CFR.....")
  #RUN CFR
  cfr_game = pyspiel.load_game("kuhn_poker", {"players": 2})
  # Create the CFR solver and run CFR iterations to compute a strategy
  cfr_solver = cfr.CFRSolver(cfr_game)
  for i in range(iterations):
    cfr_solver.evaluate_and_update_policy()
    #conv = 0
    conv = exploitability.exploitability(cfr_game, cfr_solver.average_policy())
    output_df = output_df_update(experiment_number=experiment_number, algorithm = "CFR", iteration_number = i+1, exploitability = conv, output_df = output_df)

  print("Starting MCCFR external sampling.....")
  #Run MCCFR external sampling
  mccfr_ext_game = pyspiel.load_game("kuhn_poker", {"players": 2})
  mccfr_ext_solver = external_mccfr.ExternalSamplingSolver(mccfr_ext_game, external_mccfr.AverageType.SIMPLE)
  for i in range(iterations):
    mccfr_ext_solver.iteration()
    conv = exploitability.nash_conv(mccfr_ext_game, mccfr_ext_solver.average_policy())
    output_df = output_df_update(experiment_number=experiment_number, algorithm = "MCCFR_Ext", iteration_number = i+1, exploitability = conv, output_df = output_df)

  print("Starting MCCFR outcome sampling.....")
  #Run MCCFR outcome sampling
  mccfr_out_game = pyspiel.load_game("kuhn_poker", {"players": 2})
  mccfr_out_solver = outcome_mccfr.OutcomeSamplingSolver(mccfr_out_game)
  for i in range(iterations):
    mccfr_out_solver.iteration()
    conv = exploitability.nash_conv(mccfr_out_game, mccfr_out_solver.average_policy())
    output_df = output_df_update(experiment_number=experiment_number, algorithm = "MCCFR_Out", iteration_number = i+1, exploitability = conv, output_df = output_df)

  return output_df


In [ ]:
output_df = cfr_comparison()

Starting Vanilla CFR.....
Starting MCCFR external sampling.....
Starting MCCFR outcome sampling.....


In [ ]:
output_df.tail()

,Experiment number,Algorithm,Iteration number,Time stamp,Exploitability
29995,1.0,MCCFR_Out,9996.0,2023-10-18 13:23:53.985,0.035672
29996,1.0,MCCFR_Out,9997.0,2023-10-18 13:23:53.992,0.035667
29997,1.0,MCCFR_Out,9998.0,2023-10-18 13:23:53.998,0.035357
29998,1.0,MCCFR_Out,9999.0,2023-10-18 13:23:54.004,0.035320
29999,1.0,MCCFR_Out,10000.0,2023-10-18 13:23:54.011,0.035318


In [ ]:
#ANALYSING THE DATA
import plotly.express as px

# Specify the experiment number you want to plot
experiment_number = 1.0

# Filter the data for the specific experiment
experiment_data = output_df[output_df['Experiment number'] == experiment_number]

# Create a line graph using Plotly Express
fig = px.line(experiment_data, x='Iteration number', y='Exploitability', color='Algorithm',
              markers=True, title=f'Exploitability Over Iterations for Experiment {experiment_number}')

# Show the plot
fig.show()


In [ ]:
# # Specify the experiment number you want to analyze
# experiment_number = 1.0

# # Filter the data for the specific experiment
# experiment_data = output_df[output_df['Experiment number'] == experiment_number]

# # Convert the "Time stamp" column to datetime if not already
# experiment_data['Time stamp'] = pd.to_datetime(experiment_data['Time stamp'])

# # Calculate the time spent for each iteration
# experiment_data['Time Spent'] = experiment_data.groupby('Algorithm')['Time stamp'].diff().dt.total_seconds()

# # Calculate the total time taken for each algorithm
# algorithm_total_time = experiment_data.groupby('Algorithm')['Time Spent'].sum().reset_index()

# # Create a bar chart using Plotly Express
# fig = px.bar(algorithm_total_time, x='Algorithm', y='Time Spent', title=f'Total Time Taken by Algorithms for Experiment {experiment_number}')

# # Customize the appearance of the bar chart
# fig.update_traces(marker_color='blue')

# # Show the plot
# fig.show()
